## Tóm tắt bài toán
Yêu cầu xây dựng thuật toán để đưa ra giá cả phù hợp cho từng sản phẩm. Ta sử dụng thông tin sản phẩm do Mercari cung cấp, được người bán mô tả các chi tiết của sản phẩm như tên thương hiệu sản phẩm, tên sản phẩm, tình trạng sản phẩm để dưa ra giá phù hợp nhất.

Các file dữ liệu của bài toán:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from matplotlib import pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import lightgbm as lgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Giải nén và đọc dữ liệu

Giải nén các file dữ liệu:

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
#Import tập dữ liệu train và test
train_df = pd.read_csv('train.tsv', sep = '\t')
test_df = pd.read_csv('test_stg2.tsv', sep='\t')

In [ ]:
train_df.shape

## Phân tích dữ liệu

### Tổng quan về dữ liệu

In [ ]:
train_df.head(5)

Ở tập train, mỗi 1 sản phấm sẽ có các thuộc tính: train_id, name, item_condition_id, category_name, shipping, item_description và cột price. Ở cột brand_name có nhiều giá trị NaN(giá trị rỗng)

In [ ]:
train_df.info()

Có rất nhiều giá trị null ở cột brand_name và một vài ở cột categories và item_description.

### Phân tích giá sản phẩm

In [ ]:
import seaborn as sns

sns.distplot(train_df['price'])

Ta sử dụng hàm log để đưa phân phối giá về phân phối chuẩn

In [ ]:
sns.distplot(np.log1p(train_df.price))

### Phân tích cột shipping

In [ ]:
(train_df.shipping.value_counts())*100/train_df.shape[0]

Không có chênh lệch quá lớn giữa số lượng tiền ship được trả bởi người giao và người mua

### Phân tích condition của sản phẩm

In [ ]:
(train_df.item_condition_id.value_counts())*100/train_df.shape[0]

Có rất ít sản phẩm ở tình trạng 'Fair' (4) và 'Poor' (5)

### Phân tích brand name của sản phẩm

In [ ]:
train_df.brand_name.value_counts()[:50]

### Phân tích cột categories

Cột categories có dạng general/sub1/sub2. Ta chi thành 3 cột vào dữ liệu: general, sub1 và sub2 để dễ dàng phân tích text.



In [ ]:
def split_cat(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Others', 'Others', 'Others']
    
train_df['general'], train_df['cat_1'], train_df['cat_2'] = zip(*train_df['category_name'].apply(lambda x:split_cat(x)))

test_df['general'], test_df['cat_1'], test_df['cat_2'] = zip(*test_df['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
train_df.head(5)

In [ ]:
test_df.head(5)

## Xử lý dữ liệu

### Xử lý các giá trị null

In [ ]:
def fill_missing(dataset):
    #xử lý dữ liệu null
    dataset.category_name.fillna(value="missing", inplace=True)
    dataset.brand_name.fillna(value="missing", inplace=True)
    dataset.item_description.fillna(value="missing", inplace=True)
    return (dataset)

train_df = fill_missing(train_df)
test_df = fill_missing(test_df)

In [ ]:
train_df.head(5)

### Vector hoá dữ liệu

Để thực hiện vectơ hóa cột name ta dùng mô hình BoW (Bag-of-words)

Mô hình bag-of-words là một mô hình đơn giản hóa được sử dụng trong xử lý ngôn ngữ tự nhiên và truy xuất thông tin (IR). Trong mô hình này, một văn bản (chẳng hạn như một câu hoặc một tài liệu) được biểu thị như một túi (nhiều tập hợp) các từ của nó, không tính đến ngữ pháp và thậm chí cả trật tự từ nhưng vẫn giữ tính đa nghĩa

In [ ]:
vectorizer = CountVectorizer(stop_words="english")
X_train_name = vectorizer.fit_transform(train_df['name'])
X_test_name = vectorizer.transform(test_df['name'])

Để thực hiện vectơ hóa cột item_description ta dùng mô hình TF-IDF(Term Frequency – Inverse Document Frequency)

TF-IDF là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản. Giá trị cao thể hiện độ quan trọng cao và nó phụ thuộc vào số lần từ xuất hiện trong văn bản nhưng bù lại bởi tần suất của từ đó trong tập dữ liệu

In [ ]:
tfidf = TfidfVectorizer(stop_words="english")
X_train_descp = tfidf.fit_transform(train_df['item_description'])
X_test_descp = tfidf.transform(test_df['item_description'])

**Phân loại các features còn lại thông qua LabelBinarizer**<br/>
LabelBinarizer là một thuật toán phân loại hồi quy và nhị phân có sẵn trong scikit. Nó giúp ta phân loại dữ liệu theo các label theo kiểu one-vs-all

In [ ]:
lb_brand = LabelBinarizer(sparse_output=True)
X_train_brand = lb_brand.fit_transform(train_df['brand_name'])
X_test_brand = lb_brand.transform(test_df['brand_name'])

In [ ]:
lb_item_condition_id = LabelBinarizer(sparse_output=True)
X_train_condition = lb_item_condition_id.fit_transform(train_df['item_condition_id'])
X_test_condition = lb_item_condition_id.transform(test_df['item_condition_id'])

In [ ]:
lb_shipping = LabelBinarizer(sparse_output=True)
X_train_shipping = lb_shipping.fit_transform(train_df['shipping'])
X_test_shipping = lb_shipping.transform(test_df['shipping'])

In [ ]:
lb_gen_cat = LabelBinarizer(sparse_output=True)
X_train_gen_cat = lb_gen_cat.fit_transform(train_df['general'])
X_test_gen_cat = lb_gen_cat.transform(test_df['general'])

In [ ]:
lb_cat_1 = LabelBinarizer(sparse_output=True)
X_train_cat_1 = lb_cat_1.fit_transform(train_df['cat_1'])
X_test_cat_1 = lb_cat_1.transform(test_df['cat_1'])

In [ ]:
lb_cat_2 = LabelBinarizer(sparse_output=True)
X_train_cat_2 = lb_cat_2.fit_transform(train_df['cat_1'])
X_test_cat_2 = lb_cat_2.transform(test_df['cat_1'])

Gộp các vector đặc trưng đã lấy được ở từng mục thành ma trận tổng hợp cho tập train, test

In [ ]:
from scipy.sparse import hstack
import gc

train_X = hstack([X_train_name, X_train_descp, X_train_brand, X_train_condition,
                  X_train_shipping, X_train_gen_cat, X_train_cat_1, X_train_cat_2]).tocsr()
test_X = hstack([X_test_name, X_test_descp, X_test_brand, X_test_condition,
                 X_test_shipping, X_test_gen_cat, X_test_cat_1, X_test_cat_2]).tocsr()

### Tách dữ liệu tập train ra thàn tập train và validation

* Giữ lại 80% tập train là tâp dữ liệu huấn liệu
* Dùng 20% tập train là tập validation

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_df['price'], test_size=0.2, random_state=42)

del train_X
gc.collect()

## Xây dựng mô hình

### Hàm Evaluate

In [ ]:
def rmsle(y_pred, y):
    assert len(y) == len(y_pred)
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

def evaluation(model):
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_test)
    train_score = rmsle(np.expm1(train_pred), Y_train)
    validation_score = rmsle(np.expm1(val_pred), Y_test)
    print("Training Dataset Score: {}, Validation Dataset Score: {}".format(train_score, validation_score))
    return train_score, validation_score

def train(model):
    X, Y = X_train, np.log1p(Y_train)
    model.fit(X, Y)
    evaluation(model)
    return model

### Ridge Regression Model

Ridge Regression về cơ bản cũng giống với Linear Regression truyền thống, vẫn sử dụng đến hàm mean square error nhưng có điểm khác là công thức tìm giá trị hệ số w thì bổ sung thêm ràng buộc để sao cho các hệ số w nhỏ nhất có thể đến mức gần bằng 0, nghĩa là các feature (x) ít có ảnh hưởng tới giá trị đầu ra. Ràng buộc này sử dụng công thức chuẩn hóa L2.

In [ ]:
ridge_model = Ridge(
    alpha=0.5, 
    copy_X=True, 
    fit_intercept=True, 
    max_iter=100,
    normalize=False, 
    random_state=101, 
    solver='auto', 
    tol=0.01
)
ridge_model = train(ridge_model)

### Lasso Regression

Một mô hình anh em với Ridge là Laso, thay vì Ridge sử dụng công thức chuẩn hóa L2 khi tìm hệ số w thì Laso sử dụng công thức chuẩn hóa L1. Sử dụng công thức chuẩn hóa L1 nhằm mục đích loại bớt feature ít ảnh hưởng tới đầu ra nghĩa là làm cho phần lớn các trọng số w = 0.

In [ ]:
lasso_model = Lasso(alpha=1e-06, fit_intercept=False)
lasso_model = train(lasso_model)

### LightGBM

LightGBM viết tắt của Light Gradient Boosting Machine, là mã nguồn mở xử lý thuật toán tăng cường độ dốc (Gradient Boosting) được phát triển bởi Microsoft. Gradient Boosting là một thuật toán xuất phát từ thuật toán Cây quyết định (Decision Tree), nó thực hiện việc xây dựng tuần tự nhiều Cây quyết định và tiến hành học tập.

In [ ]:
params = {
    'learning_rate': 0.75,
    'application': 'regression',
    'max_depth': 3,
    'num_leaves': 100,
    'verbosity': -1,
    'metric': 'RMSE',
}
d_train = lgb.Dataset(X_train, np.log1p(Y_train))
d_valid = lgb.Dataset(X_test, np.log1p(Y_test))
watchlist = [d_train, d_valid]

lgbm_model = lgb.train(
    params, 
    train_set=d_train, 
    num_boost_round=2200, 
    valid_sets=watchlist,
    verbose_eval=100
)

In [ ]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Model", "Train RMSLE", "Validation RMSLE"]
table.add_row(["Ridge Regression", 0.470114, 0.488450])
table.add_row(["Lasso Regression", 0.460632, 0.483396])
table.add_row(["LightGBM", 0.452536, 0.47502])
print(table)

**Nhận xét:**<br/>
* Các mô hình học máy Ridge Regression, Lasso Regression, LightGBM cũng không có nhiều sự khác biệt trên tập validation
* Vì vậy các mô hình học máy không có tính chất ảnh hưởng nhiều đến kết quả bài toán, mà kết quả phần lớn bị ảnh hưởng nhất bởi phần trích xuất đặc trưng tạo ra vector đặc trưng cho từng sản phẩm.
* Ngoài ra, ta cũng có thể áp dụng các kĩ thuật tìm kiếm để tìm kiếm bộ tham số tốt nhất cho các mô hình học máy

**Kết hợp các mô hình học máy**

In [ ]:
lasso_pred = lasso_model.predict(test_X)
ridge_pred = ridge_model.predict(test_X)
lgbm_pred = lgbm_model.predict(test_X)
test_pred = (lasso_pred + ridge_pred + lgbm_pred) / 3

**Chuẩn bị submission để nộp chấm điểm**

In [ ]:
submission: pd.DataFrame = test_df[['test_id']]
submission['price'] = np.expm1(test_pred)
submission.to_csv("submission.csv", index=False)